## 1. Data reading
Let's import some libs we need and combine files into integral dataframe.

In [16]:
import pandas as pd
import numpy as np
from scipy import stats
import glob

In [18]:
def tables_comb(path, extension, separator):
    table = pd.DataFrame()
    for file in glob.glob(f'{path}/*{extension}'):
        pretable = pd.read_csv(file, sep=separator)
        table = pd.concat([table, pretable])
    return table

olympics = tables_comb('data/athlete_events/', '.csv', ',')

Now I want to have a look on the result. Cool, seems to be a table.

In [20]:
olympics.head(5) #for just the top of this iceberg
olympics #watch it full no ads for free

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,102344,Yosmany Romero Rodrguez,M,24.0,180.0,76.0,Cuba,CUB,2000 Summer,2000.0,Summer,Sydney,Wrestling,"Wrestling Men's Middleweight, Freestyle",NaN
1,102345,Francisco Romero Arribas,M,16.0,178.0,64.0,Guatemala,GUA,1976 Summer,1976.0,Summer,Montreal,Shooting,Shooting Mixed Skeet,NaN
2,102345,Francisco Romero Arribas,M,20.0,178.0,64.0,Guatemala,GUA,1980 Summer,1980.0,Summer,Moskva,Shooting,Shooting Mixed Skeet,NaN
3,102345,Francisco Romero Arribas,M,24.0,178.0,64.0,Guatemala,GUA,1984 Summer,1984.0,Summer,Los Angeles,Shooting,Shooting Mixed Skeet,NaN
4,102345,Francisco Romero Arribas,M,32.0,178.0,64.0,Guatemala,GUA,1992 Summer,1992.0,Summer,Barcelona,Shooting,Shooting Mixed Skeet,NaN


## 2. Data filtering
Unfortunately, an evil fate  spoiled our dataset, we are now to drop the teams from the numbered countries.

In [21]:
olympics_new = olympics.replace(to_replace='\-[0-9]*', value='', regex=True)
olympics_new.Team.unique()

array(['Cuba', 'Guatemala', 'Luxembourg', 'Spain', 'Hungary',
       'United States', 'Poland', 'Switzerland', 'Soviet Union',
       'Netherlands', 'France', 'Germany', 'Mexico', 'Argentina',
       'West Germany', 'Belgium', 'Brazil', 'Norway', 'Israel', 'Peru',
       'Australia', 'Ireland', 'Canada', 'Italy', 'Enotria',
       'Equatorial Guinea', 'Dominican Republic', 'Venezuela', 'Emily',
       'Feo', 'Latvia', 'Finland', 'Sweden', 'Kenya', 'Unified Team',
       'Ukraine', 'Aruba', 'Great Britain', 'Estonia', 'New Zealand',
       'Verveine', 'Union des Socits Franais de Sports Athletiques',
       'Rhythm', 'Llanoria', 'Bahrain', 'Kyrgyzstan', 'Yugoslavia',
       'Fiji', 'Portugal', 'Philippines', 'South Africa', 'Puerto Rico',
       'Sri Lanka', 'El Salvador', 'Uruguay', 'Colombia', 'Romania',
       'East Germany', 'Senegal', 'Samoa', 'Jamaica',
       'St. Louis Southwest Turnverein #1', 'Seychelles', 'Denmark',
       'Bohemia', 'Bohemia/Great Britain', 'Turkey', 'Austri

Lots of rubbish still. Screw that, I'm getting rid of not real countries.

In [22]:
real_countries= ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Czechoslovakia', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'East Germany', 'Ecuador', 'Egypt', 'El Salvador', 'England', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Great Britain', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'North Korea', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestine', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russia', 'Rwanda', 'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Serbia and Montenegro', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'Soviet Union', 'South Africa', 'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'South Korea', 'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', 'Syria', 'Taiwan', 'Tajikistan', 'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'Unified Team', 'United Arab Emirates', 'United States', 'United States Minor Outlying Islands', 'United States Virgin Islands','Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela', 'Vietnam', 'Virgin Islands, British', 'Virgin Islands', 'Wallis and Futuna', 'West Germany', 'Yemen', 'Yugoslavia', 'Zambia', 'Zimbabwe', 'Individual Olympic Athletes', 'South Vietnam', 'Iran', 'United Arab Republic', 'France/Great Britain', 'Australia/Great Britain', 'Scotland', 'North Yemen', 'Comoros', 'Oman', 'Tanzania', 'Holland', 'Macedonia', 'Hungaria', 'Italia', 'Kosovo', ]

rubbish = olympics_new.query('Team not in @real_countries')
rubbish.Team.unique()

array(['Enotria', 'Emily', 'Feo', 'Verveine',
       'Union des Socits Franais de Sports Athletiques', 'Rhythm',
       'Llanoria', 'St. Louis Southwest Turnverein #1', 'Bohemia',
       'Bohemia/Great Britain', 'New York Turnverein, New York', 'Kitty',
       'Libellule de Paris', 'Saar', 'Australasia',
       'St. Louis Amateur Athletic Association', 'Sydney Rowing Club',
       'Marinai della nave da guerra "Varese"', 'Roma', 'Ariette',
       'Roddklubben af 1912', 'La Salle Turnverein, Chicago',
       'Cupidon Viking', 'Alain IV', 'Thames Rowing Club',
       'Federated States of Micronesia', 'Damoiselle',
       'Amateur Athletic Association', 'Jo', 'New York Athletic Club',
       'New York Athletic Club #1', 'Yeoman VII',
       'Berliner Ruderverein von 1876', 'Favorite Hammonia', 'Bolivia',
       'Greenoaks Dundee', 'Seawanhaka Boat Club', 'Gteborgs Roddklubb',
       'Aletta', 'Sif', 'Hatuey', 'Laos', 'Fantlet', 'Maid of Lebanon',
       'Nortazo', 'Umberta V', 'Socit Naut

In [23]:
olympics_filtered = olympics_new.loc[olympics_new['Team'].isin(real_countries)]

Is's also possible to compare teams' names with NOC code and replace some mistakes, but I don't have endless time.

Next, I've noticed some gender difficulties here. What is sex "G"?!

In [33]:
olympics_filtered.Sex.unique()

array(['M', 'F', 'G'], dtype=object)

In [34]:
olympics_filtered.query('Sex == "G"')

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
42,79609,Pavel Mike,G,22.0,182.0,79.0,Czechoslovakia,TCH,1972 Summer,1972.0,Summer,Munich,Handball,Handball Men's Handball,Silver
74,79630,Anatoly Mikhaylin,G,37.0,NaN,NaN,Russia,RUS,1996 Summer,1996.0,Summer,Atlanta,Sailing,Sailing Mixed Two Person Keelboat,NaN


They both must be man, I hope. Watch it fixed.

In [41]:
olympics_filtered.loc[[42, 74], 'Sex'] = 'M'
olympics_filtered.Sex.unique()

array(['M', 'F'], dtype=object)

There is also one Height outlier (340).

In [43]:
olympics_filtered[olympics_filtered['Height'] == 340]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
227,23549,Kirsty Leigh Coventry (Seward),F,28.0,340.0,64.0,Zimbabwe,ZIM,2012 Summer,2012.0,Summer,London,Swimming,Swimming Women's 200 metres Individual Medley,NaN


Other strings with her data told me her real height.

In [46]:
olympics_filtered.loc[olympics_filtered['Height']==340, ['Height']] = 176.0
olympics_filtered.loc[olympics_filtered['Name']=='Kirsty Leigh Coventry (Seward)']

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
214,23549,Kirsty Leigh Coventry (Seward),F,16.0,176.0,64.0,Zimbabwe,ZIM,2000 Summer,2000.0,Summer,Sydney,Swimming,Swimming Women's 50 metres Freestyle,NaN
215,23549,Kirsty Leigh Coventry (Seward),F,16.0,176.0,64.0,Zimbabwe,ZIM,2000 Summer,2000.0,Summer,Sydney,Swimming,Swimming Women's 100 metres Freestyle,NaN
216,23549,Kirsty Leigh Coventry (Seward),F,16.0,176.0,64.0,Zimbabwe,ZIM,2000 Summer,2000.0,Summer,Sydney,Swimming,Swimming Women's 100 metres Backstroke,NaN
217,23549,Kirsty Leigh Coventry (Seward),F,16.0,176.0,64.0,Zimbabwe,ZIM,2000 Summer,2000.0,Summer,Sydney,Swimming,Swimming Women's 200 metres Individual Medley,NaN
218,23549,Kirsty Leigh Coventry (Seward),F,20.0,176.0,64.0,Zimbabwe,ZIM,2004 Summer,2004.0,Summer,Athina,Swimming,Swimming Women's 100 metres Backstroke,Silver
219,23549,Kirsty Leigh Coventry (Seward),F,20.0,176.0,64.0,Zimbabwe,ZIM,2004 Summer,2004.0,Summer,Athina,Swimming,Swimming Women's 200 metres Backstroke,Gold
220,23549,Kirsty Leigh Coventry (Seward),F,20.0,176.0,64.0,Zimbabwe,ZIM,2004 Summer,2004.0,Summer,Athina,Swimming,Swimming Women's 200 metres Individual Medley,Bronze
221,23549,Kirsty Leigh Coventry (Seward),F,24.0,176.0,64.0,Zimbabwe,ZIM,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 100 metres Backstroke,Silver
222,23549,Kirsty Leigh Coventry (Seward),F,24.0,176.0,64.0,Zimbabwe,ZIM,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 200 metres Backstroke,Gold
223,23549,Kirsty Leigh Coventry (Seward),F,24.0,176.0,64.0,Zimbabwe,ZIM,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 200 metres Individual Medley,Silver


There is a strange age in the table. Let's correct this obvious mistake.

In [47]:
olympics_filtered.Age.unique()

array([ 24.,  16.,  20.,  32.,  36.,  49.,  26.,  22.,  25.,  29.,  34.,
        35.,  27.,  31.,  28.,  33.,  37.,  21.,  45.,  17.,  60.,  30.,
        nan,  18.,  19.,  23.,  15.,  41.,  51.,  39.,  50.,  42.,  38.,
        44.,  57.,  43.,  40.,  63.,  56.,  53.,  61.,  58.,  46.,  13.,
        48.,  47.,  14.,  54.,  73.,  59.,  52.,  74.,  65.,  55.,  69.,
        71.,  64.,  62.,  66.,  68.,  12.,  70.,  67.,  75.,  11.,  72.,
        81.,  96., 240.,  76.,  88.,  97.,  84.,  80.,  77.])

In [48]:
olympics_filtered.loc[olympics_filtered['Age']==240, ['Age']] = 24.0
olympics_filtered.loc[olympics_filtered['Name']=='Flicien Jules mile Courbet']

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
36,23459,Flicien Jules mile Courbet,M,20.0,NaN,NaN,Belgium,BEL,1908 Summer,1908.0,Summer,London,Swimming,Swimming Men's 200 metres Breaststroke,NaN
37,23459,Flicien Jules mile Courbet,M,24.0,NaN,NaN,Belgium,BEL,1912 Summer,1912.0,Summer,Stockholm,Water Polo,Water Polo Men's Water Polo,Bronze
38,23459,Flicien Jules mile Courbet,M,24.0,NaN,NaN,Belgium,BEL,1912 Summer,1912.0,Summer,Stockholm,Swimming,Swimming Men's 200 metres Breaststroke,NaN
39,23459,Flicien Jules mile Courbet,M,24.0,NaN,NaN,Belgium,BEL,1912 Summer,1912.0,Summer,Stockholm,Swimming,Swimming Men's 400 metres Breaststroke,NaN
40,23459,Flicien Jules mile Courbet,M,32.0,NaN,NaN,Belgium,BEL,1920 Summer,1920.0,Summer,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,NaN
41,23459,Flicien Jules mile Courbet,M,32.0,NaN,NaN,Belgium,BEL,1920 Summer,1920.0,Summer,Antwerpen,Swimming,Swimming Men's 400 metres Breaststroke,NaN


## 3. The youngest
The youngest male athlete was 11, female one - 11.

In [53]:
olympics_filtered.loc[olympics_filtered.Year == 1992].groupby('Sex').aggregate({'Age': 'min'})

,Age
Sex,
F,12.0
M,11.0


## 4. Mean Height


The female height was about 167, male - 178.



In [54]:
np.mean(olympics_filtered[olympics_filtered['Sex']=='M'].Height), np.std(olympics_filtered[olympics_filtered['Sex']=='M'].Height)

(178.90612463602585, 9.366103118600295)

In [55]:
np.mean(olympics_filtered[olympics_filtered['Sex']=='F'].Height), np.std(olympics_filtered[olympics_filtered['Sex']=='M'].Height)

(167.86592646904788, 9.366103118600295)

## 5. Tennis girls' height

in 2000 is 171.76 (stdev 6.48).

In [56]:
tennis = olympics_filtered.query("Sport=='Tennis'  and Sex=='F' and Year==2000")
np.mean(tennis.Height), np.std(tennis.Height)

(171.7560975609756, 6.477404708896889)

## 6. The heaviest in 2006

The heaviest guy's sport is Skeleton, lol

In [61]:
year_2006 = olympics_filtered.loc[(olympics_filtered['Sex'] == 'M') & (olympics_filtered['Year'] == 2006.0)]
heavy = year_2006[year_2006['Weight'] == max(year_2006['Weight'])]
heavy['Sport']

8102    Skeleton
Name: Sport, dtype: object

## 7. Women's gold 1980-2010

Women earned 2244 golden medals.

In [62]:
first_women = olympics_filtered[olympics_filtered['Sex']=='F'].query("Year>=1980  and Year<=2010 and Medal=='Gold'")
first_women.Medal.count()

2244

## 8. John Aalberg

John Aalberg participated 4 times in 1992 and in 1994 too.


In [65]:
aalberg = olympics_filtered.loc[olympics_filtered['Name'] == 'John Aalberg']

aalberg.loc[aalberg['Year'] == aalberg['Year'].unique()[0]].shape[0], aalberg.loc[aalberg['Year'] == aalberg['Year'].unique()[1]].shape[0]

(4, 4)

## 9. 2008 age groups

25-35 is the most represented group, 45-55 is the least represented.


In [66]:
year_2008 = olympics_filtered[olympics_filtered['Year']==2008]
age = {'[15;25)': len(year_2008.query("Age>=15  and Age<25").ID.unique()),
              '[25;35)': len(year_2008.query("Age>=25  and Age<35").ID.unique()),
              '[35;45)':len(year_2008.query("Age>=35  and Age<45").ID.unique()),
              '[45;55]':len(year_2008.query("Age>=45  and Age<=55").ID.unique())}
age

{'[15;25)': 4728, '[25;35)': 5322, '[35;45)': 626, '[45;55]': 78}

## 10. 1994 vs 2002 sports

There appeared 3 more sports in 2002.

In [67]:
year_1994 = olympics_filtered[olympics_filtered['Year'] == 1994]
year_2002 = olympics_filtered[olympics_filtered['Year'] == 2002]
year_2002['Sport'].unique().shape[0] - year_1994['Sport'].unique().shape[0]



3

## 11. Seasons and medals

I would like to start in winter.

In [71]:
winter = olympics_filtered[olympics_filtered['Season'] == 'Winter']

In [70]:
gold_w = winter[winter['Medal'] == 'Gold']
gold_w.value_counts('Team').head(3)

Team
Canada           305
Soviet Union     250
United States    166
dtype: int64

Check golden medals in winter above.

In [72]:
silver_w = winter[winter['Medal'] == 'Silver']
silver_w.value_counts('Team').head(3)

Team
United States    308
Canada           199
Norway           165
dtype: int64

Here is a table of silver winter medals.

In [73]:
bronze_w = winter[winter['Medal'] == 'Bronze']
bronze_w.value_counts('Team').head(3)

Team
Finland          215
Sweden           177
United States    161
dtype: int64

Bronze above.

The summer is coming.

In [74]:
summer = olympics_filtered[olympics_filtered['Season'] == 'Summer']

In [76]:
gold_s = summer[summer['Medal'] == 'Gold']
gold_s.value_counts('Team').head(3)

Team
United States    2363
Soviet Union      830
Germany           568
dtype: int64

In [77]:
silver_s = summer[summer['Medal'] == 'Silver']
silver_s.value_counts('Team').head(3)

Team
United States    1251
Soviet Union      635
Great Britain     594
dtype: int64

In [78]:
bronze_s = summer[summer['Medal'] == 'Bronze']
bronze_s.value_counts('Team').head(3)

Team
United States    1126
Germany           614
Soviet Union      596
dtype: int64

## 12. Height standard

In [82]:
olympics_filtered = olympics_filtered.assign(Height_z_scores=stats.zscore(olympics_filtered['Height'], nan_policy='omit'))
olympics_filtered

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Height_z_scores
0,102344,Yosmany Romero Rodrguez,M,24.0,180.0,76.0,Cuba,CUB,2000 Summer,2000.0,Summer,Sydney,Wrestling,"Wrestling Men's Middleweight, Freestyle",NaN,0.439627
1,102345,Francisco Romero Arribas,M,16.0,178.0,64.0,Guatemala,GUA,1976 Summer,1976.0,Summer,Montreal,Shooting,Shooting Mixed Skeet,NaN,0.249711
2,102345,Francisco Romero Arribas,M,20.0,178.0,64.0,Guatemala,GUA,1980 Summer,1980.0,Summer,Moskva,Shooting,Shooting Mixed Skeet,NaN,0.249711
3,102345,Francisco Romero Arribas,M,24.0,178.0,64.0,Guatemala,GUA,1984 Summer,1984.0,Summer,Los Angeles,Shooting,Shooting Mixed Skeet,NaN,0.249711
4,102345,Francisco Romero Arribas,M,32.0,178.0,64.0,Guatemala,GUA,1992 Summer,1992.0,Summer,Barcelona,Shooting,Shooting Mixed Skeet,NaN,0.249711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22581,23428,Paul Couillaud,M,30.0,NaN,NaN,France,FRA,1924 Summer,1924.0,Summer,Paris,Athletics,Athletics Men's Long Jump,NaN,NaN
22582,23429,Ismal Coulibaly,M,23.0,191.0,78.0,Mali,MLI,2016 Summer,2016.0,Summer,Rio de Janeiro,Taekwondo,Taekwondo Men's Welterweight,NaN,1.484167
22584,23431,Mamadou Coulibaly,M,NaN,NaN,NaN,Mali,MLI,1992 Summer,1992.0,Summer,Barcelona,Judo,Judo Men's HalfLightweight,NaN,NaN
22585,23432,Mohamed Coulibaly,M,18.0,NaN,NaN,Mali,MLI,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Men's 50 metres Freestyle,NaN,NaN


## 13. Scaled height

In [85]:
h_max = olympics_filtered.Height.max()
h_min = olympics_filtered.Height.min()
olympics_filtered['Height_min_max_scaled'] = (olympics_filtered.Height - h_min) / (h_max - h_min)
olympics_filtered

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Height_z_scores,Height_min_max_scaled
0,102344,Yosmany Romero Rodrguez,M,24.0,180.0,76.0,Cuba,CUB,2000 Summer,2000.0,Summer,Sydney,Wrestling,"Wrestling Men's Middleweight, Freestyle",NaN,0.439627,0.535354
1,102345,Francisco Romero Arribas,M,16.0,178.0,64.0,Guatemala,GUA,1976 Summer,1976.0,Summer,Montreal,Shooting,Shooting Mixed Skeet,NaN,0.249711,0.515152
2,102345,Francisco Romero Arribas,M,20.0,178.0,64.0,Guatemala,GUA,1980 Summer,1980.0,Summer,Moskva,Shooting,Shooting Mixed Skeet,NaN,0.249711,0.515152
3,102345,Francisco Romero Arribas,M,24.0,178.0,64.0,Guatemala,GUA,1984 Summer,1984.0,Summer,Los Angeles,Shooting,Shooting Mixed Skeet,NaN,0.249711,0.515152
4,102345,Francisco Romero Arribas,M,32.0,178.0,64.0,Guatemala,GUA,1992 Summer,1992.0,Summer,Barcelona,Shooting,Shooting Mixed Skeet,NaN,0.249711,0.515152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22581,23428,Paul Couillaud,M,30.0,NaN,NaN,France,FRA,1924 Summer,1924.0,Summer,Paris,Athletics,Athletics Men's Long Jump,NaN,NaN,NaN
22582,23429,Ismal Coulibaly,M,23.0,191.0,78.0,Mali,MLI,2016 Summer,2016.0,Summer,Rio de Janeiro,Taekwondo,Taekwondo Men's Welterweight,NaN,1.484167,0.646465
22584,23431,Mamadou Coulibaly,M,NaN,NaN,NaN,Mali,MLI,1992 Summer,1992.0,Summer,Barcelona,Judo,Judo Men's HalfLightweight,NaN,NaN,NaN
22585,23432,Mohamed Coulibaly,M,18.0,NaN,NaN,Mali,MLI,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Men's 50 metres Freestyle,NaN,NaN,NaN


## 14. Winter publication

In [88]:
def find_mean(data: pd.DataFrame, sex: str, var: str) -> pd.DataFrame:
    return data.loc[data['Sex'] == sex][var].dropna().mean()

f_weight = find_mean(winter, 'F', 'Weight')
f_age = find_mean(winter, 'F', 'Age')
f_height = find_mean(winter, 'F', 'Height')

m_weight = find_mean(winter, 'M', 'Weight')
m_age = find_mean(winter, 'M', 'Age')
m_height = find_mean(winter, 'M', 'Height')

I would import the results as a table.

In [91]:
publ_import = {'Mean height' : [m_height, f_weight, m_height - f_weight],
                 'Mean weight' : [m_weight, f_weight, m_weight - f_weight],
                 'Mean age' : [m_age, f_age, m_age - f_age]}
publ_table = pd.DataFrame(publ_import, index=['Men', 'Women', 'Difference'])

publ_table

,Mean height,Mean weight,Mean age
Men,178.679905,76.367578,25.508647
Women,59.755140,59.755140,24.014735
Difference,118.924766,16.612438,1.493912


## 15. Teams and medals

The medals do depend on the teams. I noticed it performing 11th task. Here is correlation coefficient.

In [93]:
olympics_filtered['Team'].astype('category').cat.codes.corr(olympics_filtered['Medal'].astype('category').cat.codes, method='spearman')

0.059043133518813774